# Importando Bibliotecas

In [99]:
#importar bibliotecas
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score , f1_score, precision_score, recall_score
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.model_selection import  GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import scale
%matplotlib inline

# Abrindo arquivo

In [89]:

data = pd.read_parquet('dataset_cdjr.parquet.gzip')
#Verificando as 5 primeiras linhas do arquivo a ser utilizado
display(data.head())

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,target
337,200.0,2,662.28,39.10,-188.55,0.246978,761,0.004548,3.523703,167326,33441.06,0.019804,26.850,0.009198,94.611429,7,0
266,150.0,2,0.00,149.55,-0.45,0.150000,3,0.037975,0.000000,79,78.93,0.000000,0.000,0.000000,0.000000,0,1
236,50.0,1,346.08,30.41,-102.10,2.430952,42,0.004239,3.389618,9907,18858.77,0.018351,25.525,0.095238,86.520000,4,0
274,100.0,2,0.00,43.84,-56.16,0.150968,372,0.005854,0.000000,63544,1164.11,0.000000,0.000,0.000000,0.000000,0,1
208,50.0,1,87.56,-3.05,-94.50,0.412664,229,0.004572,0.926561,50089,1786.26,0.049019,94.500,0.004367,87.560000,1,0


# Análise exploratória dos dados

In [90]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 466 entries, 337 to 92
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   feature0   466 non-null    float64
 1   feature1   466 non-null    int64  
 2   feature2   466 non-null    float64
 3   feature3   466 non-null    float64
 4   feature4   466 non-null    float64
 5   feature5   466 non-null    float64
 6   feature6   466 non-null    int64  
 7   feature7   466 non-null    float64
 8   feature8   466 non-null    float64
 9   feature9   466 non-null    int64  
 10  feature10  466 non-null    float64
 11  feature11  466 non-null    float64
 12  feature12  466 non-null    float64
 13  feature13  466 non-null    float64
 14  feature14  466 non-null    float64
 15  feature15  466 non-null    int64  
 16  target     466 non-null    int64  
dtypes: float64(12), int64(5)
memory usage: 65.5 KB


In [91]:
tipos_de_dados = pd.DataFrame(data.dtypes, columns= ['Tipos de Dados'])
tipos_de_dados.columns.name = 'Variáveis'
tipos_de_dados

Variáveis,Tipos de Dados
feature0,float64
feature1,int64
feature2,float64
feature3,float64
feature4,float64
feature5,float64
feature6,int64
feature7,float64
feature8,float64
feature9,int64


In [92]:
list_index = pd.DataFrame(data.index)
list_index

,0
0,337
1,266
2,236
3,274
4,208
...,...
461,490
462,175
463,417
464,131


In [93]:
# Contagem de valores faltantes
data.isnull().sum()

feature0     0
feature1     0
feature2     0
feature3     0
feature4     0
feature5     0
feature6     0
feature7     0
feature8     0
feature9     0
feature10    0
feature11    0
feature12    0
feature13    0
feature14    0
feature15    0
target       0
dtype: int64

In [94]:
# linhas duplicadas

data.duplicated()

337    False
266    False
236    False
274    False
208    False
       ...  
490    False
175    False
417    False
131    False
92     False
Length: 466, dtype: bool

In [95]:
# Removendo duplicadas
data = data.drop_duplicates()
data
# Observe que o numero de linha se manteve, logo não temos linhas duplicadas 

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,target
337,200.0,2,662.28,39.10,-188.55,0.246978,761,0.004548,3.523703,167326,33441.06,0.019804,26.850000,0.009198,94.611429,7,0
266,150.0,2,0.00,149.55,-0.45,0.150000,3,0.037975,0.000000,79,78.93,0.000000,0.000000,0.000000,0.000000,0,1
236,50.0,1,346.08,30.41,-102.10,2.430952,42,0.004239,3.389618,9907,18858.77,0.018351,25.525000,0.095238,86.520000,4,0
274,100.0,2,0.00,43.84,-56.16,0.150968,372,0.005854,0.000000,63544,1164.11,0.000000,0.000000,0.000000,0.000000,0,1
208,50.0,1,87.56,-3.05,-94.50,0.412664,229,0.004572,0.926561,50089,1786.26,0.049019,94.500000,0.004367,87.560000,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,1300.0,7,12133.73,244.30,-1081.90,0.395307,2642,0.008152,11.617895,324081,17343.28,0.699621,12.893827,0.030659,149.799136,81,0
175,350.0,1,0.00,-1.99,-351.99,0.684805,514,0.004000,0.000000,128499,2263.65,0.000000,0.000000,0.000000,0.000000,0,1
417,400.0,4,1100.07,-18.20,-438.60,0.748464,586,0.003683,2.508140,159109,5303.10,0.207439,62.657143,0.011945,157.152857,7,0
131,150.0,1,0.00,114.30,-35.70,0.300000,119,0.003834,0.000000,31040,334.59,0.000000,0.000000,0.000000,0.000000,0,0


# Preparação dos dados

In [96]:
display(data)

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,target
337,200.0,2,662.28,39.10,-188.55,0.246978,761,0.004548,3.523703,167326,33441.06,0.019804,26.850000,0.009198,94.611429,7,0
266,150.0,2,0.00,149.55,-0.45,0.150000,3,0.037975,0.000000,79,78.93,0.000000,0.000000,0.000000,0.000000,0,1
236,50.0,1,346.08,30.41,-102.10,2.430952,42,0.004239,3.389618,9907,18858.77,0.018351,25.525000,0.095238,86.520000,4,0
274,100.0,2,0.00,43.84,-56.16,0.150968,372,0.005854,0.000000,63544,1164.11,0.000000,0.000000,0.000000,0.000000,0,1
208,50.0,1,87.56,-3.05,-94.50,0.412664,229,0.004572,0.926561,50089,1786.26,0.049019,94.500000,0.004367,87.560000,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,1300.0,7,12133.73,244.30,-1081.90,0.395307,2642,0.008152,11.617895,324081,17343.28,0.699621,12.893827,0.030659,149.799136,81,0
175,350.0,1,0.00,-1.99,-351.99,0.684805,514,0.004000,0.000000,128499,2263.65,0.000000,0.000000,0.000000,0.000000,0,1
417,400.0,4,1100.07,-18.20,-438.60,0.748464,586,0.003683,2.508140,159109,5303.10,0.207439,62.657143,0.011945,157.152857,7,0
131,150.0,1,0.00,114.30,-35.70,0.300000,119,0.003834,0.000000,31040,334.59,0.000000,0.000000,0.000000,0.000000,0,0


In [97]:
#separa as features 
features = data.drop(['target'],1)
#separa as labels
labels = data['target']

print('Features')
print (features.head())

print ('=========')

print ('Labels')
print (labels.head())

Features
     feature0  feature1  feature2  feature3  feature4  feature5  feature6  \
337     200.0         2    662.28     39.10   -188.55  0.246978       761   
266     150.0         2      0.00    149.55     -0.45  0.150000         3   
236      50.0         1    346.08     30.41   -102.10  2.430952        42   
274     100.0         2      0.00     43.84    -56.16  0.150968       372   
208      50.0         1     87.56     -3.05    -94.50  0.412664       229   

     feature7  feature8  feature9  feature10  feature11  feature12  feature13  \
337  0.004548  3.523703    167326   33441.06   0.019804     26.850   0.009198   
266  0.037975  0.000000        79      78.93   0.000000      0.000   0.000000   
236  0.004239  3.389618      9907   18858.77   0.018351     25.525   0.095238   
274  0.005854  0.000000     63544    1164.11   0.000000      0.000   0.000000   
208  0.004572  0.926561     50089    1786.26   0.049019     94.500   0.004367   

     feature14  feature15  
337  94.61142

C:\Users\seiff\AppData\Local\Temp\ipykernel_10372\1240779292.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  features = data.drop(['target'],1)


In [103]:
#Escolhendo as melhores features com Kbest

features_list = ('feature0','feature1','feature2','feature3','feature4','feature5','feature6','feature7','feature8','feature9',
                 'feature10','feature11','feature12','feature13','feature14','feature15')

k_best_features = SelectKBest(score_func=f_classif, k='all')
k_best_features.fit_transform(features, labels)
k_best_features_scores = k_best_features.scores_
raw_pairs = zip(features_list[1:], k_best_features_scores)
ordered_pairs = list(reversed(sorted(raw_pairs, key=lambda x: x[1])))

k_best_features_final = dict(ordered_pairs[:17])
best_features = k_best_features_final.keys()
print ('')
print ("Melhores features:")
print (k_best_features_final)


Melhores features:
{'feature2': 14.966394588124537, 'feature7': 9.746241878007414, 'feature3': 8.706752075612313, 'feature11': 7.936349142264545, 'feature14': 7.007242608646202, 'feature15': 6.831655063481211, 'feature1': 6.227319816583137, 'feature6': 5.640292247378438, 'feature10': 5.31319832846744, 'feature5': 5.279750320241934, 'feature9': 3.0003680328376325, 'feature4': 2.3486029134542656, 'feature8': 1.8194343374138346, 'feature13': 1.5104071473557679, 'feature12': 1.3454617867341232}


In [104]:
#separa as features com base nas melhores features para treinamento
features = data.drop(['target','feature4','feature8','feature9','feature12','feature13'],1)


#separa as labels para treinamento
labels = data['target']

print('Features')
print (features.head())

print ('=========')

print ('Labels')
print (labels.head())

Features
     feature0  feature1  feature2  feature3  feature5  feature6  feature7  \
337     200.0         2    662.28     39.10  0.246978       761  0.004548   
266     150.0         2      0.00    149.55  0.150000         3  0.037975   
236      50.0         1    346.08     30.41  2.430952        42  0.004239   
274     100.0         2      0.00     43.84  0.150968       372  0.005854   
208      50.0         1     87.56     -3.05  0.412664       229  0.004572   

     feature10  feature11  feature14  feature15  
337   33441.06   0.019804  94.611429          7  
266      78.93   0.000000   0.000000          0  
236   18858.77   0.018351  86.520000          4  
274    1164.11   0.000000   0.000000          0  
208    1786.26   0.049019  87.560000          1  
Labels
337    0
266    1
236    0
274    1
208    0
Name: target, dtype: int64


C:\Users\seiff\AppData\Local\Temp\ipykernel_10372\774368280.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  features = data.drop(['target','feature4','feature8','feature9','feature12','feature13'],1)


In [106]:
features

,feature0,feature1,feature2,feature3,feature5,feature6,feature7,feature10,feature11,feature14,feature15
337,200.0,2,662.28,39.10,0.246978,761,0.004548,33441.06,0.019804,94.611429,7
266,150.0,2,0.00,149.55,0.150000,3,0.037975,78.93,0.000000,0.000000,0
236,50.0,1,346.08,30.41,2.430952,42,0.004239,18858.77,0.018351,86.520000,4
274,100.0,2,0.00,43.84,0.150968,372,0.005854,1164.11,0.000000,0.000000,0
208,50.0,1,87.56,-3.05,0.412664,229,0.004572,1786.26,0.049019,87.560000,1
...,...,...,...,...,...,...,...,...,...,...,...
490,1300.0,7,12133.73,244.30,0.395307,2642,0.008152,17343.28,0.699621,149.799136,81
175,350.0,1,0.00,-1.99,0.684805,514,0.004000,2263.65,0.000000,0.000000,0
417,400.0,4,1100.07,-18.20,0.748464,586,0.003683,5303.10,0.207439,157.152857,7
131,150.0,1,0.00,114.30,0.300000,119,0.003834,334.59,0.000000,0.000000,0


In [105]:
# Normalizando os dados de entrada(features)

# Gerando o novo padrão
scaler = MinMaxScaler().fit(features)
features_scale = scaler.transform(features)

print ('Features: ',features_scale.shape)
print (features_scale)

Features:  (466, 11)
[[0.01298701 0.06451613 0.01643732 ... 0.00027099 0.0439236  0.012939  ]
 [0.00974026 0.06451613 0.         ... 0.         0.         0.        ]
 [0.00324675 0.03225806 0.00858946 ... 0.00025111 0.04016713 0.00739372]
 ...
 [0.02597403 0.12903226 0.02730296 ... 0.0028385  0.07295862 0.012939  ]
 [0.00974026 0.03225806 0.         ... 0.         0.         0.        ]
 [0.00324675 0.03225806 0.         ... 0.         0.         0.        ]]
